In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import h5py
import sklearn
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split, Subset
import torchvision
import torchvision.transforms as transforms
import time

In [3]:
device = torch.device('cuda')

In [4]:
class h5ds(Dataset):
    def __init__(self, e_ds_path, p_ds_path, transform=None, target_transform=None):
        self.electron_ds_path = e_ds_path
        self.photon_ds_path = p_ds_path
        
        self.electron_ds = h5py.File(self.electron_ds_path,'r')
        self.data_e = self.electron_ds['X'][()]
        self.labels_e = self.electron_ds['y'][()]

        self.photon_ds = h5py.File(self.photon_ds_path,'r')
        self.data_p = self.photon_ds['X'][()]
        self.labels_p = self.photon_ds['y'][()]
        
        self.transform = transform
        self.target_transform = target_transform
        
        self.data = np.concatenate((self.data_e, self.data_p), axis=0)
        self.data = np.concatenate( (self.data, self.data[:,:,:,:1]), axis = 3)
        self.labels = np.concatenate((self.labels_e,self.labels_p), axis=0)
        
    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        return self.transform(self.data[idx]), self.target_transform(self.labels[idx])

In [19]:
electron_ds_path = '/kaggle/input/e2e-p1/SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5'
photon_ds_path = '/kaggle/input/e2e-p1/SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5'

transform = transforms.Compose([transforms.ToTensor(), transforms.Resize((224,224))])
target_transform = transforms.Lambda( lambda x: x.astype(int) )

ds = h5ds(electron_ds_path,photon_ds_path,transform,target_transform)

In [21]:
from torch.utils.data import Subset

# ds = Subset(ds, range(5000))

train_size = np.int32(0.8 * len(ds))
test_size = np.int32(0.1 * len(ds))
val_size = len(ds) - train_size - test_size

train_data, test_data, val_data = random_split(ds, [train_size, test_size, val_size])

train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=64, shuffle=True)

In [22]:
print(len(train_data),len(test_data),len(val_data))

398400 49800 49800


In [ ]:
# model_ResNet18 = torchvision.models.resnet18(pretrained=True).to(device)
# in_features = model_ResNet18.fc.in_features
# model_ResNet18.fc = nn.Linear(in_features, 2, device=device)

In [69]:
class Bottleneck(nn.Module):
    expansion = 4
    def __init__(self, in_channels, out_channels,downsample=None, stride=1):
        super(Bottleneck, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0)
        self.batch_norm1 = nn.BatchNorm2d(out_channels)
        
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.batch_norm2 = nn.BatchNorm2d(out_channels)
        
        self.conv3 = nn.Conv2d(out_channels, out_channels*self.expansion, kernel_size=1, stride=1, padding=0)
        self.batch_norm3 = nn.BatchNorm2d(out_channels*self.expansion)
        
        self.downsample = downsample
            
        self.stride = stride
        self.relu = nn.ReLU()
        
    def forward(self, x):
        identity = x.clone()
        x = self.relu(self.batch_norm1(self.conv1(x)))
        x = self.relu(self.batch_norm2(self.conv2(x)))
        x = self.conv3(x)
        x = self.batch_norm3(x)
        if self.downsample is not None:
            identity = self.downsample(identity)
        x+=identity
        x=self.relu(x)
        
        return x

In [70]:
class ResNet(nn.Module):
    def __init__(self, ResBlock, num_classes, num_channels=3):
        super(ResNet, self).__init__()
        self.in_channels = 64
        
        self.conv1 = nn.Conv2d(num_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.batch_norm1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.max_pool = nn.MaxPool2d(kernel_size = 3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(ResBlock, planes=64)
        self.layer2 = self._make_layer(ResBlock, planes=128, stride=2)
        self.layer3 = self._make_layer(ResBlock, planes=256, stride=2)
        self.layer4 = self._make_layer(ResBlock, planes=512, stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512*ResBlock.expansion, num_classes)
        
    def forward(self, x):
        x = self.relu(self.batch_norm1(self.conv1(x)))
        x = self.max_pool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)
        
        return x
        
    def _make_layer(self, ResBlock, planes, stride=1):
        layers=[]
        downsample = nn.Sequential(nn.Conv2d(self.in_channels, planes*ResBlock.expansion, kernel_size=1, stride=stride),
                            nn.BatchNorm2d(planes*ResBlock.expansion))
        layers.append(ResBlock(self.in_channels, planes,downsample, stride=stride))
        self.in_channels = planes*ResBlock.expansion
        layers.append(ResBlock(self.in_channels, planes))
            
        return nn.Sequential(*layers)

In [16]:
def train_loop(dataloader, model, loss_fn, optimizer):
    model.train()
    size = len(dataloader.dataset)
    running_loss = 0.
    #total_sample = 0
    for batch_i, data in enumerate(dataloader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)
#         inputs, labels = data[0], data[1]
        
        # zero the parameter gradients
        optimizer.zero_grad()
        
        # Compute prediction and loss
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        
        # Backpropagation
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        num_batch_print = 200
        if batch_i % num_batch_print == num_batch_print - 1:
            current = (batch_i + 1) * len(inputs)
            print(f'batch: [{batch_i + 1:5d}], loss: {running_loss / num_batch_print:.3f}')
            print(f'temporary loss: {loss.item():>7f} | [{current:>5d}/{size:>5d}]')
            running_loss = 0

In [73]:
def test_loop(dataloader, model, loss_fn):
    model.eval()
    #size = len(dataloader.dataset)
    num_batches = len(dataloader)
    
    test_loss, correct = 0, 0
    total = 0
    
    with torch.no_grad():
        for data in dataloader:
            # get the inputs for test dataset
            images, labels = data[0].to(device), data[1].to(device)
#             images, labels = data[0], data[1]

            
            # calculate the outputs
            outputs = model(images)
            
            # classify which class the output in
            _, predicted = torch.max(outputs.data, 1)
            #_, labels_value = torch.max(labels.data, 1)
            
            # obtain the statistics of test loss and correctness
            test_loss += loss_fn(outputs, labels).item()
            correct += (predicted == labels).sum().item() 
            #correct += (predicted == labels_value).sum().item() 
            total += labels.size(0)

    print(f"Test Error: \n Accuracy: {(100 * correct / total):>0.1f}%")
    print(f"Avg loss: {test_loss / num_batches:>8f} \n")

In [71]:
# model loss and optimizer
model = ResNet(Bottleneck,2,3).to(device)
print(model)
criterion = nn.CrossEntropyLoss()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (batch_norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (max_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (batch_norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (batch_norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
      (batch_norm3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
        (1): BatchNorm2d(256, eps=1e-05, mom

In [ ]:
# new

# criterion = nn.CrossEntropyLoss()
lr_list = [1e-2]*5
time_start = time.time()
# Pre-trained ResNet training
for t in range( len(lr_list) ):
    print(f"-------------Epoch {t+1}-------------")
    #optimizer = optim.SGD(model.parameters(), lr=lr_list[t], momentum=0.9)
    optimizer = optim.Adam(model.parameters(), lr=lr_list[t])
    train_loop(train_dataloader, model, criterion, optimizer)
    test_loop(test_dataloader, model, criterion)
print("Over")
time_end = time.time()
print("Time Consumption",time_end-time_start)

-------------Epoch 1-------------
batch: [  200], loss: 0.592
temporary loss: 0.595372 | [12800/398400]
batch: [  400], loss: 0.576
temporary loss: 0.729660 | [25600/398400]
batch: [  600], loss: 0.585
temporary loss: 0.481644 | [38400/398400]
batch: [  800], loss: 0.577
temporary loss: 0.620711 | [51200/398400]
batch: [ 1000], loss: 0.578
temporary loss: 0.540373 | [64000/398400]
batch: [ 1200], loss: 0.573
temporary loss: 0.547093 | [76800/398400]
batch: [ 1400], loss: 0.577
temporary loss: 0.547346 | [89600/398400]
batch: [ 1600], loss: 0.574
temporary loss: 0.562558 | [102400/398400]
batch: [ 1800], loss: 0.576
temporary loss: 0.586183 | [115200/398400]
batch: [ 2000], loss: 0.581
temporary loss: 0.590319 | [128000/398400]
batch: [ 2200], loss: 0.577
temporary loss: 0.505012 | [140800/398400]
batch: [ 2400], loss: 0.570
temporary loss: 0.569722 | [153600/398400]
batch: [ 2600], loss: 0.571
temporary loss: 0.673495 | [166400/398400]
batch: [ 2800], loss: 0.573
temporary loss: 0.6568